In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data VIZ
from sklearn.svm import SVC # "Support vector classifier"


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Get The Data

In [ ]:
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
train_df.info()

In [ ]:
train_df.head(1)

In [ ]:
image = np.array(train_df.drop('label', axis=1).iloc[0])

In [ ]:
image.shape

In [ ]:
# number of each label
train_df.label.value_counts()

> **Viz some results**

In [ ]:
image = image.reshape(28,28,1)
plt.imshow(image)
plt.title(f"label: {train_df.iloc[0]['label']}")

In [ ]:
plt.figure(figsize=(8,6))
for i in range(10):
    ax = plt.subplot(2, 5, i+1)
    index = np.random.randint(0, 1000)
    image = np.array(train_df.drop('label', axis=1).iloc[index])
    image = image.reshape(28,28,1)
    plt.imshow(image)
    plt.title(f"label: {train_df.iloc[index]['label']}")
    plt.axis("off")

# > **Pre_Processing**

> train_test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train_df.drop('label', axis=1).values
y = train_df.label.values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train = X_train.reshape(33600, 28, 28, 1)

In [ ]:
X_test = X_test.reshape(8400, 28, 28, 1)

In [ ]:
X_train.shape

> Rescale the images

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

> # Modeling 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout, BatchNormalization

Model Structure

In [ ]:
model= Sequential()

model.add(Conv2D(kernel_size=(3,3), filters=32, activation='relu', input_shape=(28, 28,1)))
# model.add(Conv2D(filters=30,kernel_size = (3,3),activation='relu'))

model.add(MaxPool2D(2,2))
model.add(Conv2D(filters=32,kernel_size = (3,3),activation='relu', input_shape=(28, 28,1)))

model.add(MaxPool2D(2,2))
# model.add(Conv2D(filters=32,kernel_size = (3,3),activation='relu'))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(10,activation = 'softmax'))  # the output layer

Model summary

In [ ]:
model.summary()

In [ ]:
# model compilation
model.compile(loss = 'SparseCategoricalCrossentropy', optimizer ='adam', metrics= ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [ ]:
model.fit(X_train, y_train, epochs = 30 , validation_data = (X_test, y_test), verbose=1)

In [ ]:
pd.DataFrame(model.history.history).plot()
plt.xlabel('epochs')

> **SVM**

> the accuracy is almost 97.5

In [ ]:
# svm = SVC(random_state=0)

In [ ]:
# X = train_df.drop('label', axis=1).values
# y = train_df.label.values

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# svm.fit(X_train, y_train)

In [ ]:
# predY = svm.predict(X_test)

In [ ]:
# svm.score(X_test, y_test)

# Predictions

In [ ]:
test_df.head(1)

In [ ]:
test_images = np.array(test_df)

In [ ]:
test_images.shape

In [ ]:
test_images = test_images.reshape(28000, 28, 28, 1)

In [ ]:
test_images = test_images / 255.0

In [ ]:
predictions = model.predict(test_images)
predictions = predictions.argmax(axis=1)
predictions[:5]

In [ ]:
sub_df = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
sub_df.head(2)

In [ ]:
sub_df['Label'] = predictions

In [ ]:
sub_df.head(2)

In [ ]:
# predictions value counts
sub_df.Label.value_counts()

In [ ]:
sub_df.to_csv("submission.csv", index=False)